# prepare

In [1]:
# imports
import os
import argparse
import json

import torch
import pytorch_lightning as pl
import torchmetrics
import transformers

from utils import (
    PersonaDataset,
    GenerativeCollator,
    RetrievalCollator,
    aggregate_encoder_output,
    sim_func,
)
from models import RetrievalModel, GenerativeModel

/home/stc/rybin-as/miniconda3/envs/persona/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# proxy
os.environ["http_proxy"] = "http://proxy.ad.speechpro.com:3128"
os.environ["https_proxy"] = "http://proxy.ad.speechpro.com:3128"
os.environ["ftp_proxy"] = "http://proxy.ad.speechpro.com:3128"

In [3]:
# config bert
parser = argparse.ArgumentParser()
bert_args = parser.parse_args("")
with open("configs/bert_config.json", "r") as config:
    opt = json.load(config)
vars(bert_args).update(opt)

# config gpt
parser = argparse.ArgumentParser()
gpt_args = parser.parse_args("")
with open("configs/gpt_config.json", "r") as config:
    opt = json.load(config)
vars(gpt_args).update(opt)

# pretrained model

In [4]:
with open(bert_args.special_tokens_dict, "r") as config:
    special_tokens_dict = json.load(config)

# bert tokenizer
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(
    bert_args.pretrained_bert,
    truncation_side=bert_args.truncation_side,
    padding_side=bert_args.padding_side,
)
bert_tokenizer.add_special_tokens(special_tokens_dict)

# gpt tokenizer
gpt_tokenizer = transformers.AutoTokenizer.from_pretrained(
    gpt_args.pretrained_gpt,
    truncation_side=gpt_args.truncation_side,
    padding_side=gpt_args.padding_side,
)
gpt_tokenizer.pad_token = gpt_tokenizer.eos_token
gpt_tokenizer.add_special_tokens(special_tokens_dict)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


7

In [5]:
# retrieva model
bert_ckpt = "/home/stc/persona/logs/bi_encoder/best/checkpoints/epoch=5-step=4554.ckpt"
retrieval_model = RetrievalModel.load_from_checkpoint(bert_ckpt)
retrieval_model.eval()

# generative model
gpt_ckpt = "/home/stc/persona/logs/gpt_answer/1ep.ckpt"
generative_model = GenerativeModel.load_from_checkpoint(gpt_ckpt)
generative_model.eval()

/home/stc/rybin-as/miniconda3/envs/persona/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:268: UserWarning: Attribute 'context_BERT' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['context_BERT'])`.
  rank_zero_warn(
/home/stc/rybin-as/miniconda3/envs/persona/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:268: UserWarning: Attribute 'candidat_BERT' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['candidat_BERT'])`.
  rank_zero_warn(
/home/stc/rybin-as/miniconda3/envs/persona/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:268: UserWarning: Attribute 'GPT' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['GPT'])`.
  rank_zero_warn(


GenerativeModel(
  (GPT): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50265, 1280)
      (wpe): Embedding(1024, 1280)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0): GPT2Block(
          (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (1): GPT2Block(
          (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
   

# data

In [6]:
# bert callator
bert_callator = RetrievalCollator(
    bert_tokenizer, padding=bert_args.padding, max_length=bert_args.context_len
)

# gpt callator
gpt_callator = GenerativeCollator(
    gpt_tokenizer, padding=gpt_args.padding, max_length=gpt_args.max_len
)

Using eos_token, but it is not set yet.


# inference

In [16]:
# encode functions
def encode_persona(text_batch, encoder):
    inp_persona_tokens = bert_callator.CandidateCollator(text_batch)
    vec_batch = aggregate_encoder_output(
        encoder.candidat_BERT(**inp_persona_tokens), mod=bert_args.aggregation_mod
    )
    return vec_batch


def encode_context(text_batch, encoder):
    inp_context_tokens = bert_callator.ContextCollator([text_batch])
    vec_batch = aggregate_encoder_output(
        encoder.context_BERT(**inp_context_tokens), mod=bert_args.aggregation_mod
    )
    return vec_batch

In [32]:
persona = [
    "У меня любимая работа.",
    "Я уважаю людей.",
    "У меня есть попугай.",
    "Я был в Париже.",
    "Я люблю кофе.",
    "У меня есть собака.",
    "У меня есть кошка.",
    "Я играю на гитаре.",
    "Я кассир.",
    "Я работаю в магазине.",
]
context = []

vec_persona = encode_persona(persona, retrieval_model)
# vec_context = encode_context(context, retrieval_model)
# ranks = sim_func(vec_context, vec_persona, mod=bert_args.sim_mod)[0].tolist()
# gks = sorted(list(zip(ranks, persona)), key=lambda x: x[0], reverse=True)
# print(gks)

In [35]:
user_msg = 'ты умеешь играть на гитаре?'
context.append(user_msg)
for c in context:
    print('-',c)
vec_context = encode_context(context, retrieval_model)
ranks = sim_func(vec_context, vec_persona, mod=bert_args.sim_mod)[0].tolist()
gks = sorted(list(zip(ranks, persona)), key=lambda x: x[0], reverse=True)
print("знания о персоне:", gks)
gks = [gk[1] for gk in gks[:2]]

# generate
dict_inp = [{"context": context, "gk": gks, "candidate": ""}]
gpt_inp = gpt_callator.test(dict_inp)[0]["input_ids"][:, :-2]
len_gpt_inp = gpt_inp.size()[-1]
#print(("         (" + gpt_tokenizer.batch_decode(gpt_inp)[0] + ")"))
gpt_out = generative_model.GPT.generate(
    gpt_inp,
    max_new_tokens=32,
)
gpt_answer = gpt_out
answer_raw = gpt_tokenizer.decode(gpt_answer[0], skip_special_tokens=False)
#print(("         (" + answer_raw + ")"))

# proc answer
answer = gpt_tokenizer.decode(
    gpt_answer[0, len_gpt_inp + 1 :], skip_special_tokens=True
)
context.append(answer)
print("model:", answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


- привет
- Привет, я работаю кассиром в магазине, а ты?
- у тебя есть кошки?
- Нет, у меня нет животных, а у тебя?
- ты умеешь играть на гитаре?
знания о персоне: [(4.718793869018555, 'Я кассир.'), (0.8107884526252747, 'У меня есть кошка.'), (-4.800271987915039, 'Я играю на гитаре.'), (-5.37678337097168, 'Я работаю в магазине.'), (-7.195186614990234, 'У меня есть собака.'), (-8.373382568359375, 'У меня есть попугай.'), (-11.692588806152344, 'Я уважаю людей.'), (-11.858613967895508, 'Я люблю кофе.'), (-12.386756896972656, 'У меня любимая работа.'), (-12.475095748901367, 'Я был в Париже.')]
model: Нет, но я люблю музыку и хотела бы научиться, а ты умеешь играть на каком- нибудь инструменте?


In [ ]:
# inference
# retrieva model
bert_ckpt = "/home/stc/persona/logs/bi_encoder/best/checkpoints/epoch=5-step=4554.ckpt"
retrieval_model = RetrievalModel.load_from_checkpoint(bert_ckpt)
# encode functions
def encode_persona(text_batch, encoder):
    inp_persona_tokens = bert_callator.CandidateCollator(text_batch)
    vec_batch = aggregate_encoder_output(
        encoder.candidat_BERT(**inp_persona_tokens), mod=bert_args.aggregation_mod
    )
    return vec_batch


def encode_context(text_batch, encoder):
    inp_context_tokens = bert_callator.ContextCollator([text_batch])
    print(bert_tokenizer.batch_decode(inp_context_tokens["input_ids"]))
    vec_batch = aggregate_encoder_output(
        encoder.context_BERT(**inp_context_tokens), mod=bert_args.aggregation_mod
    )
    return vec_batch
persona = [
    "У меня любимая работа.",
    "Я уважаю людей.",
    "У меня есть попугай.",
    "Я был в Париже.",
    "Я люблю кофе.",
    "У меня есть собака.",
    "У меня есть кошка.",
    "Я играю на гитаре.",
    "Я кассир.",
    "Я работаю в магазине.",
]
context = ['привет', 'привет', "как ты относишься к животным?"]

vec_persona = encode_persona(persona, retrieval_model)
vec_context = encode_context(context, retrieval_model)
ranks = sim_func(vec_context, vec_persona, mod="DotProduct")[0].tolist()
gks = sorted(list(zip(ranks, persona)), key=lambda x: x[0], reverse=True)
print(gks)